In [1]:
import pandas as pd
import numpy as np
from settings import LOCAL_DATA_PATH, ROOT_DIR
import re
from data.get_data import get_unique_values_from_columns, get_data_from_collection


2022-06-17 17:10:10.436 WARNING streamlit.state.auto_session_state: Session state does not function when running a script without `streamlit run`


In [13]:
query = {}
df = get_data_from_collection('clients', query)
df = df.reset_index()
df =  df[['_id', 'date_created', 'customer_group_id']]

def parse_date_created(df):
    df['date_created'] = pd.to_datetime(df['date_created'])
    df['month_created'] = df['date_created'].dt.month
    df['year_created'] = df['date_created'].dt.year
    df['day_created'] = df['date_created'].dt.day
    df['weekday_created'] = df['date_created'].dt.weekday
    df['hour_created'] = df['date_created'].dt.hour
    df['day_of_year_created'] = df['date_created'].dt.dayofyear
    df['month_created_str'] = df['date_created'].dt.strftime("%b")

    df['date_created'] = df['date_created'].dt.date

    return df
df = parse_date_created(df)

def aggregate_date_created(df):
    df = df[['_id', 'date_created']].groupby(['date_created']).count()
    df = df.reset_index()
    df = df.rename(columns={'_id': 'count'})
    return df

def aggregate_monthly(df):
    df = df[['_id', 'month_created']].groupby(['month_created']).count()
    df = df.reset_index()
    df = df.rename(columns={'_id': 'count'})
    return df

def aggregate_yearly(df):
    df = df[['_id', 'year_created']].groupby(['year_created']).count()
    df = df.reset_index()
    df = df.rename(columns={'_id': 'count'})
    return df

dayly_agg =  aggregate_date_created(df)
monthly_agg = aggregate_monthly(df)
yearly_agg = aggregate_yearly(df)
df

,_id,date_created,customer_group_id,month_created,year_created,day_created,weekday_created,hour_created,day_of_year_created,month_created_str
0,626596cb2f7d05734cbb20e2,2021-01-07,2,1,2021,7,3,15,7,Jan
1,626596cb2f7d05734cbb20e3,2021-01-21,0,1,2021,21,3,15,21,Jan
2,626596cb2f7d05734cbb20e4,2021-02-15,0,2,2021,15,0,9,46,Feb
3,626596cb2f7d05734cbb20e5,2021-04-08,0,4,2021,8,3,16,98,Apr
4,626596cb2f7d05734cbb20e6,2021-04-28,0,4,2021,28,2,7,118,Apr
...,...,...,...,...,...,...,...,...,...,...
405,626596cb2f7d05734cbb2277,2022-01-17,0,1,2022,17,0,16,17,Jan
406,626596cb2f7d05734cbb2278,2022-01-19,0,1,2022,19,2,22,19,Jan
407,626596cb2f7d05734cbb2279,2022-01-26,0,1,2022,26,2,1,26,Jan
408,626596cb2f7d05734cbb227a,2022-01-27,0,1,2022,27,3,9,27,Jan


In [15]:
df1 = df[['date_created', 'month_created_str', 'day_created', '_id']].groupby(['date_created', 'month_created_str', 'day_created']).count().reset_index().rename(columns={'_id': 'count'})

df1 = df1[['month_created_str','day_created','count']].set_index('day_created')

In [16]:

import plotly.express as px


fig = px.area(df1, facet_col="month_created_str", facet_col_wrap=3, color_discrete_sequence=px.colors.sequential.Plasma)
fig.show()

In [111]:
df1

,month_created,count
day_created,,
7,1,1
21,1,1
15,2,1
8,4,1
28,4,1
...,...,...
6,1,1
17,1,5
19,1,1


In [5]:
df_orders = get_data_from_collection('orders', query)
df_orders

,_id,Unnamed: 0,order_id,customer_id,date_created,date_modified,date_shipped,status_id,status,subtotal_ex_tax,...,ebay_transaction_id,option_set_id,parent_order_product_id,is_bundled_product,bin_picking_number,external_id_y,fulfillment_source,applied_discounts,product_options,configurable_fields
0,6267ec705b0da65f8048f48a,0,100,1,"Tue, 05 Mar 2019 19:17:35 +0000","Mon, 24 Jun 2019 14:10:30 +0000","Tue, 23 Apr 2019 16:11:34 +0000",5,Cancelled,49.00,...,NaN,14.0,NaN,False,NaN,NaN,NaN,[],"[{'id': 1, 'option_id': 18, 'order_product_id'...",[]
1,6267ec705b0da65f8048f48b,1,102,2,"Tue, 28 May 2019 22:50:31 +0000","Mon, 24 Jun 2019 14:10:27 +0000",NaN,5,Cancelled,149.99,...,NaN,NaN,NaN,False,NaN,NaN,NaN,[],[],[]
2,6267ec705b0da65f8048f48c,2,103,0,"Fri, 21 Jun 2019 12:35:07 +0000","Fri, 22 Nov 2019 09:14:28 +0000","Fri, 22 Nov 2019 09:14:28 +0000",10,Completed,9.99,...,NaN,NaN,NaN,False,NaN,NaN,NaN,[],[],[]
3,6267ec705b0da65f8048f48d,3,104,0,"Fri, 21 Jun 2019 20:28:22 +0000","Fri, 22 Nov 2019 09:14:29 +0000","Fri, 22 Nov 2019 09:14:29 +0000",10,Completed,0.00,...,NaN,NaN,NaN,False,NaN,NaN,NaN,[],[],[]
4,6267ec705b0da65f8048f48e,4,105,0,"Fri, 21 Jun 2019 20:28:22 +0000","Thu, 27 Jun 2019 20:11:08 +0000","Thu, 27 Jun 2019 20:11:08 +0000",10,Completed,149.00,...,NaN,NaN,NaN,False,NaN,NaN,NaN,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13412,6267ec705b0da65f804928ee,13412,13260,0,"Fri, 31 Dec 2021 17:13:42 +0000","Fri, 07 Jan 2022 20:26:12 +0000","Fri, 07 Jan 2022 20:26:12 +0000",2,Shipped,20.00,...,NaN,1238.0,NaN,False,NaN,NaN,NaN,[],"[{'id': 9513, 'option_id': 1273, 'order_produc...",[]
13413,6267ec705b0da65f804928ef,13413,13261,136855,"Fri, 31 Dec 2021 17:13:46 +0000","Fri, 07 Jan 2022 20:26:12 +0000","Fri, 07 Jan 2022 20:26:12 +0000",2,Shipped,20.00,...,NaN,1238.0,NaN,False,NaN,NaN,NaN,[],"[{'id': 9514, 'option_id': 1273, 'order_produc...",[]
13414,6267ec705b0da65f804928f0,13414,13262,14179,"Fri, 31 Dec 2021 17:13:53 +0000","Fri, 07 Jan 2022 20:26:12 +0000","Fri, 07 Jan 2022 20:26:12 +0000",2,Shipped,20.00,...,NaN,1238.0,NaN,False,NaN,NaN,NaN,[],"[{'id': 9515, 'option_id': 1273, 'order_produc...",[]
13415,6267ec705b0da65f804928f1,13415,13263,110164,"Fri, 31 Dec 2021 17:13:54 +0000","Fri, 07 Jan 2022 20:25:58 +0000","Fri, 07 Jan 2022 20:25:58 +0000",2,Shipped,20.00,...,NaN,1238.0,NaN,False,NaN,NaN,NaN,[],"[{'id': 9516, 'option_id': 1273, 'order_produc...",[]
